<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 2 / NOTE 2*

# 📝 Assignment
---

# 랜덤포레스트(Random Forests)

### 1) 캐글 대회를 이어서 진행합니다. EDA, 데이터 전처리 부분을 업데이트 하세요.
- EDA는 항상 완벽할 수 없지요 한 주간 계속 같은 데이터로 과제를 진행하므로 부족한 부분을 추가하거나 논하세요.
- (지금은 feature engineering에 너무 시간을 들이지 마세요!)
- Ordinal Encoding을 적용해 보세요.
- **(객관식) 다음 특성들 중에 순서를 고려한 Ordinal Encoding이 필요해 보이는 특성을 고르세요.**
    1. opinion_h1n1_vacc_effective
    2. state
    3. marital
    4. employment_occupation


In [1]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json
# !pip install kaggle

In [2]:
# ! pip install category_encoders
# ! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
# from category_encoders import OneHotEncoder
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline
from ipywidgets import interact
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
# from pandas_profiling import ProfileReport
import numpy as np
from sklearn.metrics import f1_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
### EDA 시작
target = 'vacc_h1n1_f'
# target = 'vacc_seas_f'
train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

In [5]:
# 훈련 / 검증 / 테스트 셋으로 분리
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train[target], random_state=2)


train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

In [6]:
# 기준 모델 설정
train[target].value_counts(normalize=True)
# 기준 모델 정확도 : 0.760935

0    0.760935
1    0.239065
Name: vacc_h1n1_f, dtype: float64

In [7]:
def engineer(df):
    """특성을 엔지니어링 하는 함수입니다."""
    
    # 높은 카디널리티를 가지는 특성을 제거합니다.
#     selected_cols = df.select_dtypes(include=['number', 'object'])
#     colnames = selected_cols.columns.tolist()
#     labels = selected_cols.nunique()
    
#     selected_features = labels[labels <= 30].index.tolist()
#     df = df[selected_features]
    
    # 새로운 특성을 생성합니다.
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)
        
    return df


train = engineer(train)
val = engineer(val)
test = engineer(test)

In [8]:
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

### EDA 끝

In [9]:
### Ordinal Encoding 적용
enc = OrdinalEncoder(handle_missing="return_nan")
X_train_ord = enc.fit_transform(X_train)
X_val_ord = enc.transform(X_val)
X_test_ord = enc.transform(X_test)

enc.category_mapping

# handle_missing="value"인 경우 opinion_h1n1_vacc_effective와 opinion_h1n1_sick_from_vacc 컬럼에서 NaN 값이 -2가 아닌 순서형 값으로 변환됨
# 이를 미리 방지하기 위해서 handle_missing="return_nan"으로 변경하여 NaN 값을 -2로 변환하도록 함

[{'col': 'opinion_h1n1_vacc_effective',
  'data_type': dtype('O'),
  'mapping': Somewhat Effective      1
  Not Very Effective      2
  Very Effective          3
  Not At All Effective    4
  Dont Know               5
  NaN                    -2
  Refused                 7
  dtype: int64},
 {'col': 'opinion_h1n1_risk',
  'data_type': dtype('O'),
  'mapping': Somewhat Low     1
  Somewhat High    2
  Very Low         3
  NaN             -2
  Very High        5
  Dont Know        6
  Refused          7
  dtype: int64},
 {'col': 'opinion_h1n1_sick_from_vacc',
  'data_type': dtype('O'),
  'mapping': Not At All Worried    1
  Not Very Worried      2
  Dont Know             3
  Somewhat Worried      4
  Very Worried          5
  NaN                  -2
  Refused               7
  dtype: int64},
 {'col': 'agegrp', 'data_type': dtype('O'), 'mapping': 18 - 34 Years         1
  35 - 44 Years         2
  65+ Years             3
  55 - 64 Years         4
  45 - 54 Years         5
  6 Months - 9 Ye

### 2) 랜덤포레스트 모델을 적용한 후의 결과를 캐글에 제출하세요.
- 랜덤포레스트를 적용하고 성능이 오히려 떨어졌을 수도 있습니다! 만약 그렇다면 이유를 본인 논리로 분석해 보세요.
- **캐글 Leaderboard의 본인 Score를 제출하세요.**

In [10]:
pipe_ord = make_pipeline(
    OrdinalEncoder(handle_missing="return_nan"), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=10, n_jobs=-1, oob_score=True)
)

pipe_ord.fit(X_train, y_train)
print('훈련 정확도', pipe_ord.score(X_train, y_train))
print('검증 정확도', pipe_ord.score(X_val, y_val))
print('f1 score', f1_score(y_val, pipe_ord.predict(X_val)))

# 훈련 정확도가 0.997인데 검증 정확도는 0.824
# 과적합으로 판단함. RandomForestClassifier의 하이퍼 파라미터를 조정하여 과적합 줄이기

훈련 정확도 0.9971532781780981
검증 정확도 0.8245759696358677
f1 score 0.5486725663716815


In [11]:
def trees(max_depth=1, min_samples_split=2, min_samples_leaf=1, n_estimators=1):
  train_score = 0
  val_score = 0

  pipe = make_pipeline(
    OrdinalEncoder(handle_missing="return_nan"), 
    SimpleImputer(), 
    RandomForestClassifier(max_depth=max_depth, 
                           min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, 
                           n_estimators=n_estimators, random_state=10, n_jobs=-1, oob_score=True)
  )

  pipe.fit(X_train, y_train)
  train_score = pipe.score(X_train, y_train)
  val_score = pipe.score(X_val, y_val)
  
  plt.bar(0, train_score, label='train')
  plt.bar(1, val_score, label='val')
  plt.text(0, train_score, s=f't:{train_score}')
  plt.text(1, val_score, s=f'v:{val_score}')
  plt.legend(loc='lower right')
  plt.show();
        
interact(trees, max_depth=(1,100,1), min_samples_split=(2,100,10), min_samples_leaf=(1,100,10),n_estimators=(10,200,10));

# max_depth=13, min_samples_split=2(default), min_samples_leaf=1(default), n_estimators=120
# 검증 정확도 : 0.8315

interactive(children=(IntSlider(value=1, description='max_depth', min=1), IntSlider(value=2, description='min_…

In [12]:
pipe_rf = make_pipeline(
    OrdinalEncoder(handle_missing="return_nan"), 
    SimpleImputer(), 
    RandomForestClassifier(max_depth=13, n_estimators=120, random_state=10, n_jobs=-1, oob_score=True)
  )

pipe_rf.fit(X_train, y_train)
print('훈련 정확도', pipe_rf.score(X_train, y_train))
print('검증 정확도', pipe_rf.score(X_val, y_val))
print('f1 score', f1_score(y_val, pipe_rf.predict(X_val)))

훈련 정확도 0.8894523025828069
검증 정확도 0.8315739532677026
f1 score 0.5520504731861199


In [13]:
X_total = pd.concat([X_train, X_val])
y_total = pd.concat([y_train, y_val])

In [14]:
# 최종 모델 학습
last_pipe = make_pipeline(
    OrdinalEncoder(handle_missing="return_nan"), 
    SimpleImputer(), 
    RandomForestClassifier(max_depth=13, n_estimators=120, random_state=10, n_jobs=-1, oob_score=True)
)

last_pipe.fit(X_total, y_total)
print('최종 정확도: ', last_pipe.score(X_total, y_total))

최종 정확도:  0.8837832708639749


In [15]:
# 최종 모델 학습 후 테스트 셋에 대한 예측 결과
y_test_pred = last_pipe.predict(X_test)

In [16]:
# 최종 결과 담은 데이터프레임 생성
d = {'Id':X_test.index, 'vacc_h1n1_f': y_test_pred}
submission = pd.DataFrame(data = d)
submission.head()

,Id,vacc_h1n1_f
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [17]:
# 제출용 파일 생성
submission.to_csv('submission.csv', index=False)

In [18]:
# !kaggle competitions submit -c prediction-of-h1n1-vaccination -f submission.csv -m "Gyeong ho Ahn, Second Submission(using Random Forest) - upgraded"

## 🔥 도전과제

### 3) 수업에 사용하지 않은 다른 종류의 [category_encoders](http://contrib.scikit-learn.org/category_encoders/)을 적용해 보고 인코더에 대해 설명하세요


In [19]:
from category_encoders import CountEncoder, TargetEncoder

In [20]:
# 훈련 데이터를 통해 인코더 비교
# 1. CountEncoder
pipe_count = make_pipeline(
    CountEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(max_depth=13, n_estimators=120, random_state=10, n_jobs=-1, oob_score=True)
  )

pipe_count.fit(X_train, y_train)
print('훈련 정확도', pipe_count.score(X_train, y_train))
print('검증 정확도', pipe_count.score(X_val, y_val))
print('f1 score', f1_score(y_val, pipe_count.predict(X_val)))

훈련 정확도 0.8913501171307416
검증 정확도 0.8322856126200925
f1 score 0.5570175438596491


In [21]:
# PolynomialEncoder만 적용 후 데이터 모양 확인해보기
enc_count = CountEncoder()
X_train_count = enc_count.fit_transform(X_train, y_train)
X_train.shape, X_train_count.shape
# 특성 수는 변함 없음

((33723, 32), (33723, 32))

In [22]:
X_train_count.head().T
# 범주형 특성 각각에 대하여 범주 종류별 개수로 범주의 값을 대체함

,20365,29138,36128,39793,32973
h1n1_concern,1.0,1.0,2.0,2.0,0.0
h1n1_knowledge,1.0,1.0,2.0,1.0,1.0
behavioral_antiviral_meds,0.0,0.0,0.0,0.0,0.0
behavioral_avoidance,0.0,1.0,1.0,1.0,0.0
behavioral_face_mask,0.0,0.0,0.0,0.0,0.0
behavioral_wash_hands,0.0,1.0,1.0,1.0,0.0
behavioral_large_gatherings,0.0,1.0,1.0,0.0,0.0
behavioral_outside_home,0.0,1.0,1.0,0.0,0.0
behavioral_touch_face,0.0,1.0,0.0,1.0,0.0
doctor_recc_h1n1,0.0,1.0,0.0,0.0,0.0


In [23]:
# 2. TargetEncoder
pipe_target = make_pipeline(
    TargetEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(max_depth=13, n_estimators=120, random_state=10, n_jobs=-1, oob_score=True)
  )

pipe_target.fit(X_train, y_train)
print('훈련 정확도', pipe_target.score(X_train, y_train))
print('검증 정확도', pipe_target.score(X_val, y_val))
print('f1 score', f1_score(y_val, pipe_target.predict(X_val)))

훈련 정확도 0.8939299587818403
검증 정확도 0.8351322500296525
f1 score 0.5738810545677498


In [24]:
# TargetEncoder만 적용 후 데이터 모양 확인해보기
enc_target = TargetEncoder()
X_train_target = enc_target.fit_transform(X_train, y_train)
X_train.shape, X_train_target.shape
# 특성 수는 변함 없음

((33723, 32), (33723, 32))

In [25]:
X_train_target.head().T
# 범주형 특성 각각에 대하여 범주 종류에 대한 타겟 값들의 평균으로 범주 값을 대체함

,20365,29138,36128,39793,32973
h1n1_concern,1.000000,1.000000,2.000000,2.000000,0.000000
h1n1_knowledge,1.000000,1.000000,2.000000,1.000000,1.000000
behavioral_antiviral_meds,0.000000,0.000000,0.000000,0.000000,0.000000
behavioral_avoidance,0.000000,1.000000,1.000000,1.000000,0.000000
behavioral_face_mask,0.000000,0.000000,0.000000,0.000000,0.000000
behavioral_wash_hands,0.000000,1.000000,1.000000,1.000000,0.000000
behavioral_large_gatherings,0.000000,1.000000,1.000000,0.000000,0.000000
behavioral_outside_home,0.000000,1.000000,1.000000,0.000000,0.000000
behavioral_touch_face,0.000000,1.000000,0.000000,1.000000,0.000000
doctor_recc_h1n1,0.000000,1.000000,0.000000,0.000000,0.000000


## 참고자료

- [Random Forests for Complete Beginners](https://victorzhou.com/blog/intro-to-random-forests/)